In [ ]:
! pip install python-dotenv elasticsearch openai tqdm

In [5]:
import json
from dotenv import load_dotenv

load_dotenv()

False

In [6]:
with open('documents.json') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
from elasticsearch import Elasticsearch

In [12]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': 'ebd949e85437', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jhFDQbH1RmCpepSf2cDRtQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Indexing

In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [14]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:27<00:00, 34.76it/s]


In [18]:
query = 'I just discovered this course. Can I still join it?'

In [16]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [19]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

## Prompt

In [20]:
def build_prompt(query, result_docs):
    prompt_template = """
        You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: 
        {context}
    """.strip()

    context = ''

    for doc in result_docs:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=result_docs).strip()

    return prompt


## Retrieval

In [21]:
from openai import OpenAI
import os

In [22]:
client = OpenAI()

In [23]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [26]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [25]:
rag("The course has started. Can I still join?")

"Yes, you can still join the course even after it has started. You are eligible to submit the homework without needing to register. However, keep in mind that there will be deadlines for submitting final projects, so it's advisable not to leave things until the last minute."

In [26]:
response = rag("How to install pgadmin using docker-compose?")

print(response)

To install pgAdmin using docker-compose, you need to create a `docker-compose.yml` file with the following configuration:

```yaml
services:
  pgdatabase:
    # Your Postgres database service configuration goes here
  pgadmin:
    image: dpage/pgadmin4
    environment:
      - PGADMIN_DEFAULT_EMAIL=admin@admin.com
      - PGADMIN_DEFAULT_PASSWORD=root
    volumes:
      - "./pgAdmin_data:/var/lib/pgadmin/sessions:rw"
    ports:
      - "8080:80"
```

In this configuration, the line `pgAdmin_data` represents a folder on your host machine that will persist pgAdmin's session configurations. Before running `docker-compose up`, ensure that the pgAdmin container has the necessary permissions to write to the `pgAdmin_data` folder. You can do this by running the following command in the terminal:

```
sudo chown -R 5050:5050 pgAdmin_data
```

After making these changes, you can execute `docker-compose up` to start your pgAdmin service.


In [22]:
client = OpenAI(
    base_url = 'http://localhost:11434',
    api_key = 'ollama'
)

In [29]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [30]:
def llm(prompt):
    response = client.chat.completions.create(
        model="llama3.1",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [ ]:
rag('the course has started. can I still join it?')